In [2]:
%%javascript
var width = window.innerWidth || document.documentElement.clientWidth || document.body.clientWidth;
var height = window.innerHeight || document.documentElement.clientHeight || document.body.clientHeight;

IPython.notebook.kernel.execute("windowSize = (" + width + "," + height + ")");
// suitable for small screens
nbpresent.mode.tree.set(
    ["app", "theme-manager", "themes", "my-theme"], 
    {
    palette: {
        "blue": { id: "blue", rgb: [0, 153, 204] },
        "black": { id: "black", rgb: [0, 0, 0] },
        "white": { id: "white", rgb: [255, 255, 255] },
        "red": { id: "red", rgb: [240, 32, 32] },
        "gray": { id: "gray", rgb: [128, 128, 128] },
    },
    backgrounds: {
        "my-background": {
            "background-color": "white"
        }
    },
    "text-base": {
        "font-family": "Georgia",
        "font-size": 2.5
    },
    rules: {
        h1: {
            "font-size": 5.5,
            color: "blue",
            "text-align": "center"
        },
        h2: {
            "font-size": 3,
            color: "blue",
            "text-align": "center"
        },
        h3: {
            "font-size": 3,
            color: "black",
        },
        "ul li": {
            "font-size": 2.5,
            color: "black"
        },
        "ul li ul li": {
            "font-size": 2.0,
            color: "black"
        },
        "code": {
            "font-size": 1.6,
        },
        "pre": {
            "font-size": 1.6,
        }
    }
});

<IPython.core.display.Javascript object>

# Returning to Dynamic Programming

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/TSproblem.png" width="800px" class="centerImg">

<p style="text-align: right; clear: right;">1</p>

# What is an Algorithm?

* An algorithm is a sequence of instructions that one must perform in order to solve a well-formulated problem.

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/AlgorithmComplexity.png" width="800px" class="centerImg">

<p style="text-align: right; clear: right;">2</p>

# Correctness

* An algorithm is correct only if it produces correct result for all input instances. 
 - If the algorithm gives an incorrect answer for one or more input instances, it is an incorrect algorithm. 
* Coin change problem
 - **Input:** an amount of money *M* in cents 
 - **Output:** the smallest number of coins
* US coin change problem

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/Coins.png" width="600px" class="centerImg">

<p style="text-align: right; clear: right;">3</p>

# US Coin Change

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/USCoinChange.png" width="600px" class="centerImg">

<p style="text-align: right; clear: right;">4</p>

# Change Problem

* Input: 
  - an amount of money M
  - an array of denominations c = (c<sub>1</sub>, c<sub>2</sub>, …,c<sub>d</sub>) <br>in order of decreasing value
* Output: the smallest number of coins

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/IncorrectBox.png" width="600px" class="centerImg">

<p style="text-align: right; clear: right;">5</p>

# Another Approach?

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/BruteForce.jpg" width="160" style="float: right; clear: right; margin: 24px;">
* Let's bring back brute force
  - Test every coin combination and see if it adds up to our target
  - Is there exhaustive search algorithm?

In [3]:
def exhaustiveChange(amount, denominations):
    bestN = 100
    count = [0 for i in xrange(len(denominations))]
    while True:
        for i, coinValue in enumerate(denominations):
            count[i] += 1
            if (count[i]*coinValue < 100):
                break
            count[i] = 0
        n = sum(count)
        if n == 0:
            break
        value = sum([count[i]*denominations[i] for i in xrange(len(denominations))])
        if (value == amount):
            if (n < bestN):
                solution = [count[i] for i in xrange(len(denominations))]
                bestN = n
    return solution

print exhaustiveChange(42, [1,5,10,20,25])

[2, 0, 0, 2, 0]


<p style="text-align: right; clear: right;">6</p>

# Other Tricks?

* A branch and bound algorithm

In [6]:
def branchAndBoundChange(amount, denominations):
    bestN = amount
    count = [0 for i in xrange(len(denominations))]
    while True:
        for i, coinValue in enumerate(denominations):
            count[i] += 1
            if (count[i]*coinValue < amount):
                break
            count[i] = 0
        n = sum(count)
        if n == 0:
            break
        if (n > bestN):
            continue
        value = sum([count[i]*denominations[i] for i in xrange(len(denominations))])
        if (value == amount):
            if (n < bestN):
                solution = [count[i] for i in xrange(len(denominations))]
                bestN = n
    return solution

print branchAndBoundChange(42, [1,5,10,20,25])

[2, 0, 0, 2, 0]


* Correct, and works well for most cases, but might be as slow as an exhaustive search for some inputs.

<p style="text-align: right; clear: right;">7</p>

# Is there another Approach?

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/Tabulate.png" width="300" style="float: right; clear: right; margin: 24px;">

* Tabulating Answers
   - If it is costly to compute the answer for a given input, then there may be advantages to caching the result of previous calculations in a table
   - This trades-off time-complexity for space
   - How could we fill in the table in the first place?
   - Run our best correct algorithm
   - Can the table itself be used to speed up the process?

<p style="text-align: right; clear: right;">8</p>

# Solutions using a Table

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/RecursiveIdea.png" width="250" style="float: right; clear: right; margin: 24px;">

* Suppose you are asked to fill-in the unknown table entry for 67&cent;
* It must differ from previous known optimal result by at most one coin…

* So what are the possibilities?

  - BestChange(67&cent;) = 25&cent; + BestChange(42&cent;), or
  - BestChange(67&cent;) = 20&cent; + BestChange(47&cent;), or
  - BestChange(67&cent;) = 10&cent; + BestChange(57&cent;), or
  - BestChange(67&cent;) = 5&cent; + BestChange(62&cent;), or
  - BestChange(67&cent;) = 1&cent; + BestChange(66&cent;)

<p style="text-align: right; clear: right;">9</p>

# A Recursive Coin-Change Algorithm

In [7]:
def RecursiveChange(M, c):
    if (M == 0):
        return [0 for i in xrange(len(c))]
    smallestNumberOfCoins = M+1
    for i in xrange(len(c)):
        if (M >= c[i]):
            thisChange = RecursiveChange(M - c[i], c)
            thisChange[i] += 1
            if (sum(thisChange) < smallestNumberOfCoins):
                bestChange = thisChange
                smallestNumberOfCoins = sum(thisChange)
    return bestChange

print RecursiveChange(42, [1,5,10,20,25])

[2, 0, 0, 2, 0]


* The only problem is... it is too slow
* Let’s see why...
<p style="text-align: right; clear: right;">10</p>

# Recursion Recalculations

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/ChangeCalls.png" width="400" style="float: right; margin-right: 100px;">

* Recursion often results in many redundant calls
* Even after only two levels of recursion 6 different change values are repeated multiple times
* How can we avoid this repetition?
* Cache precomputed results in a table!

<p style="text-align: right; clear: right;">11</p>

# Back to Table Evaluation

* When do we fill in the values of the table?
* We could do it lazily as needed… as each call to BestChange() progresses from M down to 1
* Or we could do it from the bottom-up –  tabulating all values from 1 up to M
* Thus, instead of just trying to find the minimal number of coins to change M cents, we attempt the solve the superficially harder problem of solving for the optimal change for all values from 1 to M

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/FillingTable.png" width="800px" class="centerImg">

<p style="text-align: right; clear: right;">12</p>

# Change via Dynamic Programming

In [6]:
def DPChange(M, c):
    change = [[0 for i in xrange(len(c))]]
    for m in xrange(1,M+1):
#         print change
        bestNumCoins = m+1
        for i in xrange(len(c)):
            if (m >= c[i]):
                thisChange = [x for x in change[m - c[i]]]
                thisChange[i] += 1
                if (sum(thisChange) < bestNumCoins):
                    change[m:m] = [thisChange]
                    bestNumCoins = sum(thisChange)
        print str(len(change)) + " " + str(m)
    return change[M]

print DPChange(42, [1,5,10,20,25])

2 1
3 2
4 3
5 4
7 5
8 6
9 7
10 8
11 9
14 10
15 11
16 12
17 13
18 14
20 15
21 16
22 17
23 18
24 19
28 20
29 21
30 22
31 23
32 24
35 25
36 26
37 27
38 28
39 29
41 30
42 31
43 32
44 33
45 34
48 35
49 36
50 37
51 38
52 39
55 40
56 41
57 42
[2, 0, 0, 2, 0]


* Recall, BruteForceChange( ) was O(M<sup>d</sup>)
* DPChange( ) is O(Md)

<p style="text-align: right; clear: right;">13</p>

# Dynamic Programming

* Dynamic Programming is a general technique for computing recurrence relations efficiently by storing partial or intermediate results

* Three keys to constructing a dynamic programming solution:
  1. Formulate the answer as a recurrence relation
  2. Consider all instances of the recurrence at each step
  3. Order evaluations so you will always have precomputed the needed partial results


* We'll see it again, and again

<p style="text-align: right; clear: right;">14</p>

# Next Time

* Back to sequence alignment
* Another algorithm design approach.. Divide and Conquer

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/DivideCartoon.jpg" width="700px" class="centerImg">

<p style="text-align: right; clear: right;">15</p>